# Функции получения изображений для обучающих выборок на основе данных росреестра (домами считается все, что отмечено домом на росреестре)

In [55]:
import cv2
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from selenium import webdriver
import time
from tqdm import tqdm
from selenium.common.exceptions import NoSuchElementException
import warnings

warnings.filterwarnings('ignore')

In [92]:
def new_box(box):
    l_d = [box[:,0].min(), box[:,1].min()]
    l_u = [box[:,0].min(), box[:,1].max()]
    r_d = [box[:,0].max(), box[:,1].min()]
    r_u = [box[:,0].max(), box[:,1].max()]
    return np.array([l_d, l_u, r_u, r_d])
def make_sample(img_name, img_m_name, new_name):
    hsv_min = np.array((2, 0, 0), np.uint8)
    hsv_max = np.array((255, 255, 255), np.uint8)
    img = cv2.imread(img_name)
    img_m = cv2.imread(img_m_name)
    hsv = cv2.cvtColor( img_m, cv2.COLOR_BGR2HSV )
    thresh = cv2.inRange( hsv, hsv_min, hsv_max )
    contours0, hierarchy = cv2.findContours( thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    result = pd.DataFrame()
    for cnt in contours0:
        rect = cv2.minAreaRect(cnt)
        box = cv2.boxPoints(rect)
        box = np.int0(box)
        n_box = new_box(box)
        area = int(rect[1][0]*rect[1][1])
        
        if area > 200 and area < 300000:
            cv2.drawContours(img,[n_box],0,(0,0,255),1)
            result = result.append(pd.DataFrame({
                'name':[new_name],
                'left_d x':[n_box[0,0]],
                'left_d y':[n_box[0,1]],
                'right_u x':[n_box[2,0]],
                'right_u y':[n_box[2,1]],
                'is house':['True']
            }))

    return img, result
def all_img(image_names):
    result_log = pd.DataFrame()
    for i in range(image_names.shape[0]):
        img_name, img_m_name = image_names.iloc[i]['img name'], image_names.iloc[i]['img m name']
        img, r = make_sample(img_name, img_m_name, 'IMG'+str(i)+'.png')
        cv2.imwrite('IMG'+str(i)+'.jpg', img)
        result_log = result_log.append(r)
    result_log.set_index('name').to_csv('log.csv')
def get_img_names(f, t):
    result = pd.DataFrame()
    for i in range(f, t + 1):
        result = result.append(pd.DataFrame({
            'img name':[str(i)+'_1.png'],
            'img m name':[str(i)+'.png']
        }))
    return result
def schreenshot_maker(driver, points):
    for i in range(points.shape[0]):
        point = points.iloc[i]
        lat, lon = point['lat'], point['lon']
        url_m = 'https://pkk.rosreestr.ru/#/layers/'+str(lat)+','+str(lon)+'/17/@fzn9rog5'
        url = 'https://pkk.rosreestr.ru/#/layers/'+str(lat)+','+str(lon)+'/17/@e8ia5x8l'
        driver.get(url_m)
        time.sleep(3)
        driver.save_screenshot (str(i)+".png" )
        driver.get(url)
        time.sleep(3)
        driver.save_screenshot (str(i)+"_1.png" )
def poit_maker(lat_begin, lon_begin, n_i, n_j, step_lat, step_lon):
    result = {'lat':[],'lon':[]}
    for i in range(n_i):
        for j in range(n_j):
            result['lat'].append(lat_begin + i*step_lat)
            result['lon'].append(lon_begin + j*step_lon)
    return pd.DataFrame(result)

In [88]:
points = poit_maker(55.59872088357345, 37.704957037223494, 7, 7, 0.005, 0.015)

In [58]:
driver_path = 'C:/Users/Иван/Downloads/chromedriver_win32_new/chromedriver.exe'
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
driver = webdriver.Chrome(driver_path, chrome_options=None)

In [89]:
schreenshot_maker(driver, points)

In [93]:
names = get_img_names(0, 48)
all_img(names)